library(glmmTMB)

rm(list=ls())
DF<-read.table(file="cjz-2022-0107supplb.csv",header=T,sep=",",na.strings="NA")
DF$YYYY<-as.factor(DF$YYYY)
DF$MM2<-as.factor(DF$MM2)
DF$GRID<-as.factor(DF$GRID)
DF$HANDLER_FIELD<-as.factor(DF$HANDLER_FIELD)
DF$ID<-as.factor(DF$ID)
DF$REPRO1<-as.factor(DF$REPRO1)

########SCALING VARIABLES##########################
DF$T_LOADz<-scale(DF$T_LOAD)
DF$MASSz  <-scale(DF$MASS_FIELD)
DF$Tempz  <-scale(DF$MeanTemp)
DF$Precipz<-scale(DF$TotPrecip)
DF$DENSdz <-scale(DF$DENSd)
DF$F_LOADz<-scale(DF$F_LOAD)
DF$B_LOADz<-scale(DF$B_LOAD)
DF$M_LOADz<-scale(DF$M_LOAD)

############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
############################## MAIN analysis ####################################
#fit HURDLE model
fit_h_nbinom2<-glmmTMB(T_LOAD~DENSyc+MM2+Toosoon+GRID+Tempz+Precipz+SEXc+AGEc+REPROc+MASSz+DENSdz+F_LOADz+B_LOADz+M_LOADz+
                         (1|YYYY)+(1|ID)+(1|HANDLER_FIELD),
                       family="truncated_nbinom2"(link="log"),
                       data = DF,
                       ziformula=~DENSyc+MM2+Toosoon+GRID+Tempz+Precipz+SEXc+AGEc+REPROc+MASSz+DENSdz+F_LOADz+B_LOADz+M_LOADz+
                         (1|YYYY)+(1|ID)+(1|HANDLER_FIELD))
summary(fit_h_nbinom2)



#re-fit HURDLE model, but with a different variable for reproductive status
REPRO_model<-glmmTMB(T_LOAD~DENSyc+MM2+Toosoon+GRID+Tempz+Precipz+SEXc+AGEc+REPRO1+MASSz+DENSdz+F_LOADz+B_LOADz+M_LOADz+
                         (1|YYYY)+(1|ID)+(1|HANDLER_FIELD),
                       family="truncated_nbinom2"(link="log"),
                       data = DF,
                       ziformula=~DENSyc+MM2+Toosoon+GRID+Tempz+Precipz+SEXc+AGEc+REPRO1+MASSz+DENSdz+F_LOADz+B_LOADz+M_LOADz+
                         (1|YYYY)+(1|ID)+(1|HANDLER_FIELD))
summary(REPRO_model)


###########Distance moved model#################
m.DIST<-lmer(DISTz~DENSyc+MM2+GRID+SEQ_OF+TRAPz+TIMEz+SEXc+AGEc+REPROc+MASSz+DENSdz+FLEAc+TICKc+BOTFLYc+MITEc+TICKc:FLEAc+(1|YYYY)+(1|ID),data=OF_DATA)
 rptGaussian(DISTz~DENSyc+MM2+GRID+SEQ_OF+TRAPz+TIMEz+SEXc+AGEc+REPROc+MASSz+DENSdz+FLEAc+TICKc+BOTFLYc+MITEc+FLEAc:TICKc+(1|YYYY)+(1|ID),grname=c("YYYY", "ID"),npermut = 50, data = OF_DATA)
hist(resid(m.DIST))
summary(m.DIST)
#graph for flea/tick interaction,
pred <- effect("FLEAc:TICKc",m.DIST,xlevels=4, confidence.level = 0.95)
resp <- summary(pred, type="response")
graph <- data.frame(x=rownames(resp$effect), fit=resp$effect, lower=resp$lower, upper=resp$upper)
graph$x <- as.numeric(as.character(graph$x))
head(graph)
x11(5,5)
par(las=1)
plot(DISTz~FLEAc,OF_DATA,ylab="Distance moved in the open-field test (sd units)",xlab="",ylim=c(-0.5,1.5),col="white",xaxt="n")
axis(1,seq(-0.5,0.5),labels = c("Fleas not present","Fleas present"))
polygon(c(graph$x,rev(graph$x)),c(graph$lower..0.5,rev(graph$upper..0.5)),col=rgb(0,0,1,0.1),border = FALSE)
polygon(c(graph$x,rev(graph$x)),c(graph$lower.0.5,rev(graph$upper.0.5)),col=rgb(1,0,0,0.1),border = FALSE)
lines(graph$x, graph$fit..0.5,type="l", lwd=2, col="blue")
lines(graph$x, graph$lower..0.5, type="l", col="blue", lty="dotted")
lines(graph$x, graph$upper..0.5, type="l", col="blue", lty="dotted")
lines(graph$x, graph$fit.0.5,type="l", lwd=2, col="darkred")
lines(graph$x, graph$lower.0.5, type="l", col="darkred", lty="dotted")
lines(graph$x, graph$upper.0.5, type="l", col="darkred", lty="dotted")
legend(x="topleft",c("Ticks not present","Ticks present"),cex=0.9, col = c("blue","darkred"),pch=c(16,16))
#########################
DIST.coeff <-cbind(summary(m.DIST)$coefficients[,1],"±",summary(m.DIST)$coefficients[,2],summary(m.DIST)$coefficients[,5])